# CatBoostClassifier

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 2000) # since we have a lot of features
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("data"))

['test.csv', 'train.csv', 'sample_submission.csv']


In [4]:
''' Import data '''

DATA_DIR = 'data'
train_file = os.path.join(DATA_DIR, 'train.csv')
test_file = os.path.join(DATA_DIR, 'test.csv')
submission_file = os.path.join(DATA_DIR, 'sample_submission.csv')

train = pd.read_csv(train_file)
test = pd.read_csv(test_file)

X = train.drop(columns=['y', 'sample_id'])
y = train.y
X_test = test.drop(columns=['sample_id'])

X_all = X.append(X_test)

In [5]:
''' Gather all preprocessing functions '''

from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import bisect
from scipy.stats import shapiro
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM


''' 1 - divide to training and holdout datasets '''

X_train, X_hold, y_train, y_hold = train_test_split(X, y, random_state=3, test_size=0.3)

''' 2 - define the same validation scheme for all models '''

skf = RepeatedStratifiedKFold(n_repeats=5, random_state=17)


''' Function 1 - delete all columns with NaN or Inf '''
columns_to_keep = X_all.replace([np.inf, -np.inf], np.nan).dropna(axis=1).columns

def del_nan(x):
    return x[columns_to_keep]
delete_nan = FunctionTransformer(del_nan, validate=False)


''' Function 2 - delete NaN-only columns '''

na_cols = X_all[X_all.isna().any()[lambda x: x].index].isna().agg(['sum', 'count']).T

all_na_cols = na_cols[na_cols['sum'] == na_cols['count']]

def drop_all_na(X):
    ''' Here we drop all columns which consist of NaN only '''
    idx = all_na_cols.index.values
    return X.drop(columns=idx)

drop_all_na_f = FunctionTransformer(drop_all_na, validate=False)


''' Function 3 - replace NaN '''

has_na_cols = na_cols.T.drop(columns=all_na_cols.index.values).T
has_na_cols['perc'] = has_na_cols['sum']/has_na_cols['count']

'''def replace_na(X, add_boolean=True, threshold=0.3):
    Here we apply two filling strategies
            for two types of columns with na:
            % na > threshold - fill with mean
            % na < threshold - fill with 0.
        Plus we add additional columns showing
            if the value was na before transformation.
    X_copy = X.copy()
    cols_less = has_na_cols[has_na_cols['perc'] < threshold].index.values
    cols_more = has_na_cols[has_na_cols['perc'] >= threshold].index.values
    X_less = X_copy[cols_less]
    X_less = X_less.fillna(X_less.mean())
    X_copy[cols_less] = X_less
    X_copy[cols_more] = X_more
    if add_boolean:
        was_na = X[has_na_cols.index.values].isna().astype('int64')
        was_na.columns = was_na.columns.map(lambda x: x + '_na')
        X_copy = pd.concat([X_copy, was_na], axis=1)
    return X_copy'''
    
# I should have checked columns for categorical features before

def replace_na(X, add_boolean=True):
    ''' Here we apply mean filling strategy.
        And add additional columns showing if the values
        was na before transformation.
    '''
    X_copy = X.copy()
    cols = has_na_cols.index.values
    X_c = X_copy[cols]
    X_c = X_c.fillna(X_all.replace([np.inf, -np.inf], np.nan).mean())
    X_copy[cols] = X_c
    if add_boolean:
        was_na = X[has_na_cols.index.values].isna().astype('int64')
        was_na.columns = was_na.columns.map(lambda x: x + '_na')
        X_copy = pd.concat([X_copy, was_na], axis=1)
    return X_copy

replace_na_f = FunctionTransformer(replace_na, validate=False)


''' Function 4 - replace inf '''

pos_inf_cols = X_all.fillna(0).replace([np.inf], 
                                       np.nan).isna().any()[lambda x: x].index
pos_inf_cols = X_all[pos_inf_cols].replace([np.inf], 
                                           np.nan).isna().agg(['sum', 'count']).T
neg_inf_cols = X_all.fillna(0).replace([-np.inf],
                                       np.nan).isna().any()[lambda x: x].index

def fill_inf(X, add_boolean=True, multiplier=2):
    '''
    Here we apply filling strategy for infinite values:
        we replace it with absolute maximum value * multiplier.
    Additionally, we new columns showing if the value
        was infinite before filling.
    '''
    X_copy = X.copy()
    X_pos = X_copy[pos_inf_cols.index.values]
    to_fill = X_pos.replace(np.inf, np.nan).abs().max() * multiplier + 1.0
    X_pos = X_pos.replace(np.inf, np.nan).fillna(to_fill)
    X_copy[pos_inf_cols.index.values] = X_pos
    if add_boolean:
        was_inf = X[pos_inf_cols.index.values].replace(np.inf, np.nan).isna().astype('int64')
        was_inf.columns = was_inf.columns.map(lambda x: x + '_inf')
        X_copy = pd.concat([X_copy, was_inf], axis=1)
    return X_copy

fill_inf_f = FunctionTransformer(fill_inf, validate=False)


''' First block of data cleaning is gathered in one function '''

def pre_proc(X):
    return fill_inf(replace_na(drop_all_na(X)))


''' Function and class 5 - delete zero-variance features '''

w = fill_inf(replace_na(drop_all_na(X_all))).var()
zero_var_cols = w[np.isclose(w, 0)]

def drop_zero_var(X):
    idx = zero_var_cols.index.values
    return X.drop(columns=idx)

drop_zero_var_f = FunctionTransformer(drop_zero_var)

class Drop_zero_var:
    def __init__(self):
        self.zero_var_cols = []
        self.is_fitted = False
        
    def fit(self, X, y=None):
        w = X.var()
        self.zero_var_cols = w[np.isclose(w, 0.0)].index.values
        self.is_fitted = True
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        return X_copy.drop(columns=self.zero_var_cols)
    
    def fit_transform(self, X, y=None):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)

print(Drop_zero_var().fit_transform(pre_proc(X_train), y_train).shape)

''' Function and class 6 - drop highly correlated features '''

'''
# Here is the function
print(drop_zero_var(pre_proc(X_all)).shape)
corr_matrix = pre_proc(X_all).corr().abs()
print(corr_matrix.shape)

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(len(to_drop))

'''

class Drop_corr:
    def __init__(self, threshold=0.95):
        self.to_drop = []
        self.is_fitted = False
        self.threshold = threshold
        
    def fit(self, X, y=None):
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        self.to_drop = [column for column in upper.columns if any(upper[column] > self.threshold)]
        self.is_fitted = True
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        return X_copy.drop(columns=self.to_drop)
    
    def fit_transform(self, X, y=None):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)

print(Drop_corr().fit_transform(pre_proc(X_train), y_train).shape)
    
''' Class 7 - encode categorical labels '''

class LE_df:
    def __init__(self, threshold=5):
        self.le_dict = {}
        self.threshold = threshold
        
    def fit(self, X, y=None):
        X_copy = X.copy()
        counts = X_copy.apply(lambda x: x.value_counts().shape[0])
        categorical = X_copy[counts[counts <= self.threshold].index]
        for col in categorical.columns:
            le = LabelEncoder()
            le.fit(categorical[col])
            self.le_dict[col] = le
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        for col in self.le_dict.keys():
            X_copy[col] = X_copy[col].map(lambda s: -100500 if s not in \
                                          self.le_dict[col].classes_ else s)
            if np.any(X_copy[col]==-100500):
                le_classes = self.le_dict[col].classes_.tolist()
                le_classes.append(-100500)
                self.le_dict[col].classes_ = np.asarray(le_classes)
            X_copy[col] = self.le_dict[col].transform(X_copy[col])
        return X_copy
    
    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

print(LE_df().fit_transform(pre_proc(X_train), y_train).shape)
    
''' Class 8 - target mean encode categorical labels '''

class LOOTME_df:
    ''' Class performs leave-one-out target mean encoding '''
    def __init__(self, threshold=5):
        self.threshold = threshold
        self.tme_dict = {}
        self.is_fitted = False
        
    def fit(self, X, y):
        X_copy = X.copy()
        y_copy = y.copy()
        counts = X_copy.apply(lambda x: x.value_counts().shape[0])
        categorical = X_copy[counts[counts <= self.threshold].index]
        for col in categorical.columns:
            df = pd.concat((X_copy[col], y_copy), axis=1)
            df.columns = ['col', 'y']
            tme = {}
            for value in df['col'].unique():
                y_val = df[df['col'] == value]['y'].values
                col_val = df[df['col'] == value]['col'].values
                loo_mean = np.mean((np.full_like(col_val, y_val.sum()) - y_val) / y_val.shape[0])
                tme[value] = loo_mean
            self.tme_dict[col] = tme
        self.is_fitted = True
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        for col in self.tme_dict.keys():
            X_copy[col] = X_copy[col].map(lambda s: -100500 if s not in self.tme_dict[col].keys() else s)
            self.tme_dict[col][-100500] = 0.0
            X_copy[col] = X_copy[col].map(self.tme_dict[col])
        return X_copy
        
    
    def fit_transform(self, X, y):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)
    
print(LOOTME_df().fit_transform(pre_proc(X_train), y_train).shape)


''' Class 9 - One Hot Encoding of categorical features '''

class OneHot_df:
    def __init__(self, threshold=5, drop_collinear=False):
        self.threshold = threshold
        self.drop_collinear = drop_collinear
        self.categorical = []
        self.ohe = None
        self.column_names = None
        self.is_fitted = False
        
    def fit(self, X, y=None):
        X_copy = X.copy()
        counts = X_copy.apply(lambda x: x.value_counts().shape[0])
        X_cat = X_copy[counts[(counts <= self.threshold) & (counts > 1)].index]
        ohe = OneHotEncoder(handle_unknown='ignore')
        ohe.fit(X_cat)
        self.categorical = X_cat.columns
        self.ohe = ohe
        self.is_fitted = True
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_cat = X_copy.copy()[self.categorical]
        X_cat_tr = self.ohe.transform(X_cat).toarray()
        if not self.column_names:
            column_names = []
            for i, col in enumerate(self.categorical):
                column_names.extend([col + '_' + str(s) for s in self.ohe.categories_[i]])
            self.column_names = column_names
        X_cat_tr = pd.DataFrame(X_cat_tr, columns=self.column_names, index=X_copy.index)
        X_other = X_copy.drop(columns=self.categorical)
        X_copy = pd.concat([X_other, X_cat_tr], axis=1)
        if self.drop_collinear:
            to_drop = pd.Series(self.column_names, 
        index=self.column_names).apply(lambda x: x.endswith('_0'))[lambda x: x].index
            X_copy.drop(columns=to_drop)
        return X_copy
    
    def fit_transform(self, X, y=None):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)

print(OneHot_df().fit(pre_proc(X_train)).transform(pre_proc(X_hold)).shape)

''' Class 10 - find best normalizing transformation '''

def get_shapiro_p(X):
    _, p = shapiro(X)
    return p

class Find_Trans:
    def __init__(self, threshold=10):
        self.threshold = threshold
        self.pvals = []
        self.numerical = []
        self.is_fitted = False
        self.mms = MinMaxScaler()
        self.trans_dict = {'no': lambda x: x,
                           'x^2': lambda x: np.power(x, 2), 
                           'x^3': lambda x: np.power(x, 3), 
                           'log(x)': lambda x: np.log(x), 
                           #'exp(x)': lambda x: np.exp(x/10), 
                           'sqrt(x)': lambda x: np.sqrt(x)}
    
    def fit(self, X, y=None):
        X_copy = X.copy()
        counts = X_copy.apply(lambda x: x.value_counts().shape[0])
        self.numerical = counts[counts > self.threshold].index.values
        X_numer = X_copy[self.numerical]
        idx = X_numer.index
        X_numer = pd.DataFrame(self.mms.fit_transform(X_numer) + 0.01,
                               columns=self.numerical,
                               index=idx)
        pvals = pd.DataFrame(X_numer.apply(get_shapiro_p), columns=['no'])
        for name in self.trans_dict.keys():
            if name not in pvals.columns:
                pvals[name] = X_numer.apply(self.trans_dict[name]).apply(get_shapiro_p)
        self.pvals = pvals
        self.is_fitted = True
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_numer = X_copy[self.numerical]
        idx = X_numer.index
        X_numer = pd.DataFrame(self.mms.transform(X_numer) + 0.01,
                               columns=self.numerical,
                               index=idx)
        best_transform = self.pvals.idxmax(axis=1)
        best_transform = best_transform.map(self.trans_dict).to_dict()
        X_numer = X_numer.apply(best_transform)
        X_numer = X_numer.fillna(X_numer.min()-1)
        for col in self.numerical:
            X_copy[col] = X_numer[col]
        return X_copy
    
    def fit_transform(self, X, y=None):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)

print(Find_Trans().fit(pre_proc(X_train), y_train).transform(pre_proc(X_hold)).shape)


''' Class 11 - find and drop outliers '''

class DetectOut_df:
    def __init__(self, method='IsolationForest', out_fraction=0.05, 
                 drop=True, random_state=17,
                 kernel='rbf', shrink=True):
        self.method = method
        self.out_fraction = out_fraction
        self.drop = drop
        self.is_fitted = False
        self.random_state = random_state
        self.kernel = kernel
        self.shrink = shrink
        self.model = None
        self.train_set = []
        
    def fit(self, X, y=None):
        if self.method == 'IsolationForest':
            model = IsolationForest(random_state=self.random_state,
                        n_estimators=100,
                        max_features=0.9,
                        contamination=self.out_fraction, #
                        n_jobs=-1,
                        behaviour='new')
        if self.method == 'OneClassSVM':
            model = OneClassSVM(kernel=self.kernel,
                     nu=self.out_fraction,
                     shrinking=self.shrink,
                     gamma='scale'
                    )
        model.fit(X, y)
        self.model = model
        self.is_fitted = True
        self.train_set = X.copy()
        return self
    
    def transform(self, X, y):
        ''' We shouldn't predict any outliers for test set '''
        X_copy = X.copy()
        X_copy['outlier'] = 1
        X_copy['y'] = y.copy()
        if self.train_set.equals(X):
            X_copy['outlier'] = self.model.predict(X)
        if self.drop:
            X_copy = X_copy[X_copy['outlier'] == 1]
            X_copy = X_copy.drop(columns=['outlier'])
        y_copy = X_copy['y']
        X_copy = X_copy.drop(columns=['y'])
        return X_copy, y_copy
    
    def fit_transform(self, X, y):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)
    
print(DetectOut_df().fit_transform(pre_proc(X_train), y_train)[0].shape)

(766, 1777)
(766, 932)
(766, 1908)
(766, 1908)
(329, 2518)
(329, 1908)
(727, 1908)


# Базовый CatBoost

In [6]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

cat = CatBoostClassifier(learning_rate=0.001, 
                                                iterations=500,
                                                loss_function='CrossEntropy',
                                                custom_metric='AUC:hints=skip_train~false',
                                                eval_metric='AUC:hints=skip_train~false',
                                                random_state=17,
                                                rsm=0.1,
                                                bootstrap_type='Bernoulli',
                                                subsample=0.9,
                                                verbose=100)
pipe_cat = Pipeline([('drop_na', delete_nan),
                     ('cat', cat)])

pipe_cat.fit(X_train, y_train)

#scores = cross_val_score(pipe_cat, X_train, y_train, cv=skf, scoring='roc_auc')

print('Train ROC AUC: %.4f' % roc_auc_score(y_train, pipe_cat.predict_proba(X_train)[:,1]))
#print('Cross-validation ROC AUC: mean %.4f, std %.4f' % (scores.mean(), scores.std()))
print('Holdout ROC AUC: %.4f' % roc_auc_score(y_hold, pipe_cat.predict_proba(X_hold)[:,1]))

0:	learn: 0.8060754	total: 30.2ms	remaining: 15.1s
100:	learn: 0.8924454	total: 3.38s	remaining: 13.4s
200:	learn: 0.8949478	total: 6.55s	remaining: 9.75s
300:	learn: 0.8966257	total: 9.98s	remaining: 6.6s
400:	learn: 0.8992141	total: 13s	remaining: 3.21s
499:	learn: 0.9012720	total: 16.1s	remaining: 0us
Train ROC AUC: 0.9013
Holdout ROC AUC: 0.8191


## После трансформации фич

In [9]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

CORR_THRESHOLD = 0.94
CAT_THRESHOLD = 5 # <- and less unique values make feature categorical

cat = CatBoostClassifier(learning_rate=0.001, 
                                                iterations=500,
                                                loss_function='CrossEntropy',
                                                custom_metric='AUC:hints=skip_train~false',
                                                eval_metric='AUC:hints=skip_train~false',
                                                random_state=17,
                                                rsm=0.1,
                                                bootstrap_type='Bernoulli',
                                                subsample=0.9,
                                                verbose=100)
pipe_cat = Pipeline([('pre_proc', FunctionTransformer(pre_proc, validate=False)),
                      ('drop_zero_var', Drop_zero_var()),
                      ('drop_corr', Drop_corr(threshold=CORR_THRESHOLD)),
                      ('lootme', LOOTME_df(threshold=CAT_THRESHOLD)),
                      ('transform', Find_Trans(threshold=CAT_THRESHOLD)),
                     ('cat', cat)])

pipe_cat.fit(X_train, y_train)

#scores = cross_val_score(pipe_cat, X_train, y_train, cv=skf, scoring='roc_auc')

print('Train ROC AUC: %.4f' % roc_auc_score(y_train, pipe_cat.predict_proba(X_train)[:,1]))
#print('Cross-validation ROC AUC: mean %.4f, std %.4f' % (scores.mean(), scores.std()))
print('Holdout ROC AUC: %.4f' % roc_auc_score(y_hold, pipe_cat.predict_proba(X_hold)[:,1]))

0:	learn: 0.7889527	total: 27ms	remaining: 13.5s
100:	learn: 0.8905381	total: 2.92s	remaining: 11.6s
200:	learn: 0.8959516	total: 6.2s	remaining: 9.22s
300:	learn: 0.8985831	total: 8.78s	remaining: 5.8s
400:	learn: 0.9017381	total: 11.4s	remaining: 2.81s
499:	learn: 0.9048930	total: 14s	remaining: 0us
Train ROC AUC: 0.9049
Holdout ROC AUC: 0.8198


In [10]:
pipe_cat.fit(X, y)

submission = pd.read_csv(submission_file)
submission['y'] = pipe_cat.predict_proba(X_test)[:,1]
submission.to_csv('submission_cat_transform.csv', index=False)

0:	learn: 0.7877042	total: 26.2ms	remaining: 13.1s
100:	learn: 0.8896817	total: 3.84s	remaining: 15.2s
200:	learn: 0.8913231	total: 6.82s	remaining: 10.2s
300:	learn: 0.8931157	total: 10.2s	remaining: 6.75s
400:	learn: 0.8954426	total: 13s	remaining: 3.2s
499:	learn: 0.8980190	total: 16.4s	remaining: 0us


## Финальная модель - 0.88414

In [11]:
%%time
''' Let's finally play with Catboost '''

from sklearn.feature_selection import SelectKBest, f_classif
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

CORR_THRESHOLD = 0.94
CAT_THRESHOLD = 5 # <- and less unique values make feature categorical
#OUT_FRACTION = 0.05 # <- percent of outliers
#K = 400
L2_LEAF_REG = 0.5 # <- regularization coefficient for CatBoost
DEPTH = 10 # <- max tree depth for Catboost, can't be >10, or kernel will just die

cat = CatBoostClassifier(learning_rate=0.001, 
                         iterations=500,
                         loss_function='CrossEntropy',
                         l2_leaf_reg=L2_LEAF_REG,
                         depth=DEPTH,
                         custom_metric='AUC:hints=skip_train~false', #CPU-only
                         eval_metric='AUC:hints=skip_train~false', #CPU-only
                         random_state=17,
                         rsm=0.1, #CPU-only
                         bootstrap_type='Bernoulli',
                         subsample=0.9,
                         #task_type='GPU',
                         verbose=100)
pipe_cat = Pipeline([('pre_proc', FunctionTransformer(pre_proc, validate=False)),
                      ('drop_zero_var', Drop_zero_var()),
                      ('drop_corr', Drop_corr(threshold=CORR_THRESHOLD)),
                      ('lootme', LOOTME_df(threshold=CAT_THRESHOLD)),
                      ('transform', Find_Trans(threshold=CAT_THRESHOLD)),
                      #('feat_sel', SelectKBest(f_classif, K)),
                      ('cat', cat)])

pipe_cat.fit(X_train, y_train)

#scores = cross_val_score(pipe_cat, X_train, y_train, cv=skf, scoring='roc_auc')

print('Train ROC AUC: %.4f' % roc_auc_score(y_train, pipe_cat.predict_proba(X_train)[:,1]))
#print('Cross-validation ROC AUC: mean %.4f, std %.4f' % (scores.mean(), scores.std()))
print('Holdout ROC AUC: %.4f' % roc_auc_score(y_hold, pipe_cat.predict_proba(X_hold)[:,1]))

0:	learn: 0.8183259	total: 278ms	remaining: 2m 18s
100:	learn: 0.9601761	total: 33.6s	remaining: 2m 12s
200:	learn: 0.9649228	total: 1m	remaining: 1m 29s
300:	learn: 0.9673679	total: 1m 31s	remaining: 1m
400:	learn: 0.9700640	total: 2m 12s	remaining: 32.6s
499:	learn: 0.9722581	total: 2m 53s	remaining: 0us
Train ROC AUC: 0.9723
Holdout ROC AUC: 0.8433
CPU times: user 7min 27s, sys: 38.3 s, total: 8min 5s
Wall time: 3min 11s


In [12]:
pipe_cat.fit(X, y)

submission = pd.read_csv(submission_file)
submission['y'] = pipe_cat.predict_proba(X_test)[:,1]
submission.to_csv('submission_cat_modified.csv', index=False)

0:	learn: 0.8581760	total: 400ms	remaining: 3m 19s
100:	learn: 0.9522713	total: 42.5s	remaining: 2m 47s
200:	learn: 0.9557651	total: 1m 28s	remaining: 2m 10s
300:	learn: 0.9581447	total: 2m 11s	remaining: 1m 27s
400:	learn: 0.9607246	total: 2m 53s	remaining: 42.9s
499:	learn: 0.9631394	total: 3m 27s	remaining: 0us


# Попробуем обучить поглубже

In [8]:
%%time
''' Let's finally play with Catboost '''

from sklearn.feature_selection import SelectKBest, f_classif
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

CORR_THRESHOLD = 0.94
CAT_THRESHOLD = 5 # <- and less unique values make feature categorical
#OUT_FRACTION = 0.05 # <- percent of outliers
#K = 400
L2_LEAF_REG = 0.5 # <- regularization coefficient for CatBoost
DEPTH = 12 # <- max tree depth for Catboost, can't be >10, or kernel will just die

cat = CatBoostClassifier(learning_rate=0.001, 
                         iterations=500,
                         loss_function='CrossEntropy',
                         l2_leaf_reg=L2_LEAF_REG,
                         depth=DEPTH,
                         custom_metric='AUC:hints=skip_train~false', #CPU-only
                         eval_metric='AUC:hints=skip_train~false', #CPU-only
                         random_state=17,
                         rsm=0.1, #CPU-only
                         bootstrap_type='Bernoulli',
                         subsample=0.9,
                         #task_type='GPU',
                         verbose=1)
pipe_cat = Pipeline([('pre_proc', FunctionTransformer(pre_proc, validate=False)),
                      ('drop_zero_var', Drop_zero_var()),
                      ('drop_corr', Drop_corr(threshold=CORR_THRESHOLD)),
                      ('lootme', LOOTME_df(threshold=CAT_THRESHOLD)),
                      ('transform', Find_Trans(threshold=CAT_THRESHOLD)),
                      #('feat_sel', SelectKBest(f_classif, K)),
                      ('cat', cat)])

pipe_cat.fit(X_train, y_train)

#scores = cross_val_score(pipe_cat, X_train, y_train, cv=skf, scoring='roc_auc')

print('Train ROC AUC: %.4f' % roc_auc_score(y_train, pipe_cat.predict_proba(X_train)[:,1]))
#print('Cross-validation ROC AUC: mean %.4f, std %.4f' % (scores.mean(), scores.std()))
print('Holdout ROC AUC: %.4f' % roc_auc_score(y_hold, pipe_cat.predict_proba(X_hold)[:,1]))

0:	learn: 0.8372878	total: 1.23s	remaining: 10m 16s
1:	learn: 0.9261853	total: 2.77s	remaining: 11m 30s
2:	learn: 0.9283148	total: 3.95s	remaining: 10m 54s
3:	learn: 0.9532711	total: 5.3s	remaining: 10m 57s
4:	learn: 0.9618611	total: 6.76s	remaining: 11m 9s
5:	learn: 0.9637684	total: 8.16s	remaining: 11m 12s
6:	learn: 0.9620117	total: 9.4s	remaining: 11m 2s
7:	learn: 0.9637326	total: 10.8s	remaining: 11m 4s
8:	learn: 0.9659446	total: 12.1s	remaining: 11m
9:	learn: 0.9658550	total: 13.4s	remaining: 10m 57s
10:	learn: 0.9685940	total: 14.7s	remaining: 10m 55s
11:	learn: 0.9690529	total: 16s	remaining: 10m 49s
12:	learn: 0.9684076	total: 17.3s	remaining: 10m 48s
13:	learn: 0.9671456	total: 18.6s	remaining: 10m 45s
14:	learn: 0.9680563	total: 19.7s	remaining: 10m 37s
15:	learn: 0.9684506	total: 21s	remaining: 10m 35s
16:	learn: 0.9671385	total: 21.1s	remaining: 10m
17:	learn: 0.9692465	total: 22.8s	remaining: 10m 10s
18:	learn: 0.9687446	total: 24.6s	remaining: 10m 22s
19:	learn: 0.9683933

156:	learn: 0.9812568	total: 3m 33s	remaining: 7m 45s
157:	learn: 0.9813787	total: 3m 34s	remaining: 7m 44s
158:	learn: 0.9815508	total: 3m 36s	remaining: 7m 43s
159:	learn: 0.9815149	total: 3m 37s	remaining: 7m 42s
160:	learn: 0.9816799	total: 3m 39s	remaining: 7m 41s
161:	learn: 0.9817372	total: 3m 40s	remaining: 7m 40s
162:	learn: 0.9818519	total: 3m 42s	remaining: 7m 39s
163:	learn: 0.9819380	total: 3m 43s	remaining: 7m 38s
164:	learn: 0.9818878	total: 3m 44s	remaining: 7m 36s
165:	learn: 0.9818304	total: 3m 46s	remaining: 7m 35s
166:	learn: 0.9818089	total: 3m 47s	remaining: 7m 34s
167:	learn: 0.9816655	total: 3m 49s	remaining: 7m 32s
168:	learn: 0.9818663	total: 3m 50s	remaining: 7m 31s
169:	learn: 0.9822678	total: 3m 51s	remaining: 7m 29s
170:	learn: 0.9820957	total: 3m 52s	remaining: 7m 28s
171:	learn: 0.9822176	total: 3m 54s	remaining: 7m 26s
172:	learn: 0.9823467	total: 3m 55s	remaining: 7m 25s
173:	learn: 0.9823682	total: 3m 57s	remaining: 7m 24s
174:	learn: 0.9823539	total:

309:	learn: 0.9850786	total: 7m 3s	remaining: 4m 19s
310:	learn: 0.9851288	total: 7m 5s	remaining: 4m 18s
311:	learn: 0.9851861	total: 7m 6s	remaining: 4m 16s
312:	learn: 0.9852005	total: 7m 7s	remaining: 4m 15s
313:	learn: 0.9852148	total: 7m 8s	remaining: 4m 14s
314:	learn: 0.9852005	total: 7m 10s	remaining: 4m 12s
315:	learn: 0.9852148	total: 7m 11s	remaining: 4m 11s
316:	learn: 0.9853152	total: 7m 12s	remaining: 4m 9s
317:	learn: 0.9853654	total: 7m 14s	remaining: 4m 8s
318:	learn: 0.9852865	total: 7m 15s	remaining: 4m 6s
319:	learn: 0.9853080	total: 7m 16s	remaining: 4m 5s
320:	learn: 0.9853511	total: 7m 18s	remaining: 4m 4s
321:	learn: 0.9853367	total: 7m 19s	remaining: 4m 2s
322:	learn: 0.9852937	total: 7m 20s	remaining: 4m 1s
323:	learn: 0.9853295	total: 7m 21s	remaining: 3m 59s
324:	learn: 0.9853367	total: 7m 23s	remaining: 3m 58s
325:	learn: 0.9854013	total: 7m 24s	remaining: 3m 57s
326:	learn: 0.9853367	total: 7m 24s	remaining: 3m 55s
327:	learn: 0.9853295	total: 7m 26s	rema

462:	learn: 0.9883626	total: 10m 32s	remaining: 50.6s
463:	learn: 0.9883339	total: 10m 33s	remaining: 49.2s
464:	learn: 0.9883196	total: 10m 35s	remaining: 47.8s
465:	learn: 0.9883554	total: 10m 36s	remaining: 46.5s
466:	learn: 0.9883482	total: 10m 38s	remaining: 45.1s
467:	learn: 0.9883913	total: 10m 39s	remaining: 43.7s
468:	learn: 0.9884988	total: 10m 41s	remaining: 42.4s
469:	learn: 0.9885562	total: 10m 42s	remaining: 41s
470:	learn: 0.9885849	total: 10m 43s	remaining: 39.6s
471:	learn: 0.9885777	total: 10m 44s	remaining: 38.3s
472:	learn: 0.9885705	total: 10m 46s	remaining: 36.9s
473:	learn: 0.9885490	total: 10m 47s	remaining: 35.5s
474:	learn: 0.9885203	total: 10m 49s	remaining: 34.2s
475:	learn: 0.9885060	total: 10m 50s	remaining: 32.8s
476:	learn: 0.9885060	total: 10m 51s	remaining: 31.4s
477:	learn: 0.9885203	total: 10m 53s	remaining: 30.1s
478:	learn: 0.9885705	total: 10m 54s	remaining: 28.7s
479:	learn: 0.9886135	total: 10m 55s	remaining: 27.3s
480:	learn: 0.9888000	total: 1

Ну, на холде скор вроде повыше, но попробуем, че б не засабмитить.

In [9]:
pipe_cat.fit(X, y)

submission = pd.read_csv(submission_file)
submission['y'] = pipe_cat.predict_proba(X_test)[:,1]
submission.to_csv('submission_cat_modified_2.csv', index=False)

0:	learn: 0.8694904	total: 1.21s	remaining: 10m 2s
1:	learn: 0.8961948	total: 2.54s	remaining: 10m 31s
2:	learn: 0.8954883	total: 3.8s	remaining: 10m 28s
3:	learn: 0.8979452	total: 5.08s	remaining: 10m 30s
4:	learn: 0.9009961	total: 6.32s	remaining: 10m 25s
5:	learn: 0.9097236	total: 7.5s	remaining: 10m 17s
6:	learn: 0.9147956	total: 8.79s	remaining: 10m 19s
7:	learn: 0.9233262	total: 10.2s	remaining: 10m 26s
8:	learn: 0.9295792	total: 11.6s	remaining: 10m 30s
9:	learn: 0.9316793	total: 12.8s	remaining: 10m 25s
10:	learn: 0.9321714	total: 14s	remaining: 10m 21s
11:	learn: 0.9397653	total: 15.2s	remaining: 10m 19s
12:	learn: 0.9393998	total: 16.5s	remaining: 10m 16s
13:	learn: 0.9422223	total: 17.8s	remaining: 10m 16s
14:	learn: 0.9462468	total: 19.1s	remaining: 10m 18s
15:	learn: 0.9483803	total: 20.4s	remaining: 10m 17s
16:	learn: 0.9490236	total: 21.7s	remaining: 10m 17s
17:	learn: 0.9518759	total: 23.1s	remaining: 10m 18s
18:	learn: 0.9519919	total: 24.4s	remaining: 10m 18s
19:	lear

155:	learn: 0.9681305	total: 3m 30s	remaining: 7m 44s
156:	learn: 0.9682500	total: 3m 32s	remaining: 7m 43s
157:	learn: 0.9682078	total: 3m 33s	remaining: 7m 42s
158:	learn: 0.9682606	total: 3m 35s	remaining: 7m 41s
159:	learn: 0.9683098	total: 3m 36s	remaining: 7m 40s
160:	learn: 0.9684925	total: 3m 37s	remaining: 7m 38s
161:	learn: 0.9684925	total: 3m 39s	remaining: 7m 38s
162:	learn: 0.9684258	total: 3m 42s	remaining: 7m 39s
163:	learn: 0.9685171	total: 3m 43s	remaining: 7m 38s
164:	learn: 0.9686296	total: 3m 45s	remaining: 7m 37s
165:	learn: 0.9686366	total: 3m 47s	remaining: 7m 37s
166:	learn: 0.9684750	total: 3m 48s	remaining: 7m 36s
167:	learn: 0.9685242	total: 3m 50s	remaining: 7m 35s
168:	learn: 0.9682957	total: 3m 51s	remaining: 7m 33s
169:	learn: 0.9686507	total: 3m 53s	remaining: 7m 33s
170:	learn: 0.9685663	total: 3m 55s	remaining: 7m 32s
171:	learn: 0.9685277	total: 3m 56s	remaining: 7m 30s
172:	learn: 0.9684820	total: 3m 57s	remaining: 7m 28s
173:	learn: 0.9684047	total:

308:	learn: 0.9724187	total: 7m 22s	remaining: 4m 33s
309:	learn: 0.9723870	total: 7m 25s	remaining: 4m 33s
310:	learn: 0.9723554	total: 7m 27s	remaining: 4m 32s
311:	learn: 0.9723202	total: 7m 29s	remaining: 4m 30s
312:	learn: 0.9722921	total: 7m 30s	remaining: 4m 29s
313:	learn: 0.9724081	total: 7m 32s	remaining: 4m 28s
314:	learn: 0.9724608	total: 7m 34s	remaining: 4m 27s
315:	learn: 0.9726506	total: 7m 36s	remaining: 4m 25s
316:	learn: 0.9726928	total: 7m 37s	remaining: 4m 24s
317:	learn: 0.9727385	total: 7m 38s	remaining: 4m 22s
318:	learn: 0.9727631	total: 7m 40s	remaining: 4m 21s
319:	learn: 0.9727631	total: 7m 41s	remaining: 4m 19s
320:	learn: 0.9727702	total: 7m 43s	remaining: 4m 18s
321:	learn: 0.9727877	total: 7m 44s	remaining: 4m 17s
322:	learn: 0.9728334	total: 7m 46s	remaining: 4m 15s
323:	learn: 0.9727948	total: 7m 46s	remaining: 4m 13s
324:	learn: 0.9728686	total: 7m 47s	remaining: 4m 11s
325:	learn: 0.9728299	total: 7m 49s	remaining: 4m 10s
326:	learn: 0.9729283	total:

461:	learn: 0.9770021	total: 10m 55s	remaining: 53.9s
462:	learn: 0.9770021	total: 10m 56s	remaining: 52.5s
463:	learn: 0.9770056	total: 10m 58s	remaining: 51.1s
464:	learn: 0.9770302	total: 10m 59s	remaining: 49.7s
465:	learn: 0.9770267	total: 11m 1s	remaining: 48.3s
466:	learn: 0.9770443	total: 11m 2s	remaining: 46.8s
467:	learn: 0.9770056	total: 11m 4s	remaining: 45.4s
468:	learn: 0.9770126	total: 11m 5s	remaining: 44s
469:	learn: 0.9770759	total: 11m 7s	remaining: 42.6s
470:	learn: 0.9771286	total: 11m 8s	remaining: 41.2s
471:	learn: 0.9771849	total: 11m 9s	remaining: 39.7s
472:	learn: 0.9772341	total: 11m 10s	remaining: 38.3s
473:	learn: 0.9772587	total: 11m 12s	remaining: 36.9s
474:	learn: 0.9772552	total: 11m 13s	remaining: 35.5s
475:	learn: 0.9772868	total: 11m 15s	remaining: 34s
476:	learn: 0.9773008	total: 11m 16s	remaining: 32.6s
477:	learn: 0.9773852	total: 11m 17s	remaining: 31.2s
478:	learn: 0.9773957	total: 11m 18s	remaining: 29.8s
479:	learn: 0.9773782	total: 11m 20s	re

In [10]:
#0.89165
!kaggle competitions submit -c skill-task -f submission_cat_modified_2.csv -m "Глубина 12"


100%|██████████████████████████████████████| 5.89k/5.89k [00:00<00:00, 39.1kB/s]
Successfully submitted to Skill task

In [12]:
%%time
''' Let's finally play with Catboost '''

from sklearn.feature_selection import SelectKBest, f_classif
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

CORR_THRESHOLD = 0.94
CAT_THRESHOLD = 5 # <- and less unique values make feature categorical
#OUT_FRACTION = 0.05 # <- percent of outliers
#K = 400
L2_LEAF_REG = 0.5 # <- regularization coefficient for CatBoost
DEPTH = 12 # <- max tree depth for Catboost, can't be >10, or kernel will just die

cat = CatBoostClassifier(learning_rate=0.001, 
                         iterations=400,
                         loss_function='CrossEntropy',
                         l2_leaf_reg=L2_LEAF_REG,
                         depth=DEPTH,
                         custom_metric='AUC:hints=skip_train~false', #CPU-only
                         eval_metric='AUC:hints=skip_train~false', #CPU-only
                         random_state=17,
                         rsm=0.1, #CPU-only
                         bootstrap_type='Bernoulli',
                         subsample=0.9,
                         #task_type='GPU',
                         verbose=1)
pipe_cat = Pipeline([('pre_proc', FunctionTransformer(pre_proc, validate=False)),
                      ('drop_zero_var', Drop_zero_var()),
                      ('drop_corr', Drop_corr(threshold=CORR_THRESHOLD)),
                      ('lootme', LOOTME_df(threshold=CAT_THRESHOLD)),
                      ('transform', Find_Trans(threshold=CAT_THRESHOLD)),
                      #('feat_sel', SelectKBest(f_classif, K)),
                      ('cat', cat)])

pipe_cat.fit(X_train, y_train)

#scores = cross_val_score(pipe_cat, X_train, y_train, cv=skf, scoring='roc_auc')

print('Train ROC AUC: %.4f' % roc_auc_score(y_train, pipe_cat.predict_proba(X_train)[:,1]))
#print('Cross-validation ROC AUC: mean %.4f, std %.4f' % (scores.mean(), scores.std()))
print('Holdout ROC AUC: %.4f' % roc_auc_score(y_hold, pipe_cat.predict_proba(X_hold)[:,1]))

0:	learn: 0.8372878	total: 1.3s	remaining: 8m 38s
1:	learn: 0.9261853	total: 2.9s	remaining: 9m 37s
2:	learn: 0.9283148	total: 4.26s	remaining: 9m 23s
3:	learn: 0.9532711	total: 5.71s	remaining: 9m 25s
4:	learn: 0.9618611	total: 7.15s	remaining: 9m 25s
5:	learn: 0.9637684	total: 8.6s	remaining: 9m 24s
6:	learn: 0.9620117	total: 9.87s	remaining: 9m 14s
7:	learn: 0.9637326	total: 11.3s	remaining: 9m 12s
8:	learn: 0.9659446	total: 12.6s	remaining: 9m 8s
9:	learn: 0.9658550	total: 14s	remaining: 9m 4s
10:	learn: 0.9685940	total: 15.3s	remaining: 9m 1s
11:	learn: 0.9690529	total: 16.7s	remaining: 9m
12:	learn: 0.9684076	total: 18.1s	remaining: 8m 58s
13:	learn: 0.9671456	total: 19.6s	remaining: 9m
14:	learn: 0.9680563	total: 20.8s	remaining: 8m 54s
15:	learn: 0.9684506	total: 22.1s	remaining: 8m 51s
16:	learn: 0.9671385	total: 22.2s	remaining: 8m 20s
17:	learn: 0.9692465	total: 23.8s	remaining: 8m 24s
18:	learn: 0.9687446	total: 25.4s	remaining: 8m 28s
19:	learn: 0.9683933	total: 26.6s	rema

156:	learn: 0.9812568	total: 3m 53s	remaining: 6m 1s
157:	learn: 0.9813787	total: 3m 54s	remaining: 5m 59s
158:	learn: 0.9815508	total: 3m 56s	remaining: 5m 58s
159:	learn: 0.9815149	total: 3m 58s	remaining: 5m 57s
160:	learn: 0.9816799	total: 4m	remaining: 5m 56s
161:	learn: 0.9817372	total: 4m 1s	remaining: 5m 55s
162:	learn: 0.9818519	total: 4m 3s	remaining: 5m 54s
163:	learn: 0.9819380	total: 4m 5s	remaining: 5m 52s
164:	learn: 0.9818878	total: 4m 6s	remaining: 5m 51s
165:	learn: 0.9818304	total: 4m 8s	remaining: 5m 50s
166:	learn: 0.9818089	total: 4m 10s	remaining: 5m 49s
167:	learn: 0.9816655	total: 4m 12s	remaining: 5m 48s
168:	learn: 0.9818663	total: 4m 13s	remaining: 5m 47s
169:	learn: 0.9822678	total: 4m 15s	remaining: 5m 46s
170:	learn: 0.9820957	total: 4m 17s	remaining: 5m 44s
171:	learn: 0.9822176	total: 4m 18s	remaining: 5m 43s
172:	learn: 0.9823467	total: 4m 20s	remaining: 5m 41s
173:	learn: 0.9823682	total: 4m 22s	remaining: 5m 41s
174:	learn: 0.9823539	total: 4m 24s	re

309:	learn: 0.9850786	total: 7m 37s	remaining: 2m 12s
310:	learn: 0.9851288	total: 7m 39s	remaining: 2m 11s
311:	learn: 0.9851861	total: 7m 40s	remaining: 2m 9s
312:	learn: 0.9852005	total: 7m 42s	remaining: 2m 8s
313:	learn: 0.9852148	total: 7m 43s	remaining: 2m 6s
314:	learn: 0.9852005	total: 7m 44s	remaining: 2m 5s
315:	learn: 0.9852148	total: 7m 46s	remaining: 2m 3s
316:	learn: 0.9853152	total: 7m 47s	remaining: 2m 2s
317:	learn: 0.9853654	total: 7m 49s	remaining: 2m
318:	learn: 0.9852865	total: 7m 50s	remaining: 1m 59s
319:	learn: 0.9853080	total: 7m 51s	remaining: 1m 57s
320:	learn: 0.9853511	total: 7m 53s	remaining: 1m 56s
321:	learn: 0.9853367	total: 7m 54s	remaining: 1m 54s
322:	learn: 0.9852937	total: 7m 56s	remaining: 1m 53s
323:	learn: 0.9853295	total: 7m 57s	remaining: 1m 52s
324:	learn: 0.9853367	total: 7m 58s	remaining: 1m 50s
325:	learn: 0.9854013	total: 8m	remaining: 1m 48s
326:	learn: 0.9853367	total: 8m	remaining: 1m 47s
327:	learn: 0.9853295	total: 8m 2s	remaining: 

In [14]:
pipe_cat.fit(X, y)

submission = pd.read_csv(submission_file)
submission['y'] = pipe_cat.predict_proba(X_test)[:,1]
submission.to_csv('submission_cat_modified_3.csv', index=False)

0:	learn: 0.8694904	total: 1.82s	remaining: 12m 8s
1:	learn: 0.8961948	total: 3.88s	remaining: 12m 51s
2:	learn: 0.8954883	total: 5.32s	remaining: 11m 44s
3:	learn: 0.8979452	total: 6.71s	remaining: 11m 4s
4:	learn: 0.9009961	total: 8.09s	remaining: 10m 39s
5:	learn: 0.9097236	total: 9.4s	remaining: 10m 17s
6:	learn: 0.9147956	total: 10.9s	remaining: 10m 9s
7:	learn: 0.9233262	total: 12.6s	remaining: 10m 16s
8:	learn: 0.9295792	total: 14s	remaining: 10m 8s
9:	learn: 0.9316793	total: 16s	remaining: 10m 22s
10:	learn: 0.9321714	total: 17.6s	remaining: 10m 23s
11:	learn: 0.9397653	total: 19s	remaining: 10m 14s
12:	learn: 0.9393998	total: 20.3s	remaining: 10m 3s
13:	learn: 0.9422223	total: 22.1s	remaining: 10m 10s
14:	learn: 0.9462468	total: 23.8s	remaining: 10m 10s
15:	learn: 0.9483803	total: 25.7s	remaining: 10m 16s
16:	learn: 0.9490236	total: 27.3s	remaining: 10m 14s
17:	learn: 0.9518759	total: 28.6s	remaining: 10m 7s
18:	learn: 0.9519919	total: 29.9s	remaining: 9m 58s
19:	learn: 0.9531

156:	learn: 0.9682500	total: 3m 45s	remaining: 5m 49s
157:	learn: 0.9682078	total: 3m 47s	remaining: 5m 47s
158:	learn: 0.9682606	total: 3m 48s	remaining: 5m 46s
159:	learn: 0.9683098	total: 3m 49s	remaining: 5m 44s
160:	learn: 0.9684925	total: 3m 51s	remaining: 5m 43s
161:	learn: 0.9684925	total: 3m 52s	remaining: 5m 42s
162:	learn: 0.9684258	total: 3m 54s	remaining: 5m 40s
163:	learn: 0.9685171	total: 3m 55s	remaining: 5m 39s
164:	learn: 0.9686296	total: 3m 56s	remaining: 5m 37s
165:	learn: 0.9686366	total: 3m 58s	remaining: 5m 35s
166:	learn: 0.9684750	total: 3m 59s	remaining: 5m 34s
167:	learn: 0.9685242	total: 4m 1s	remaining: 5m 32s
168:	learn: 0.9682957	total: 4m 2s	remaining: 5m 31s
169:	learn: 0.9686507	total: 4m 3s	remaining: 5m 29s
170:	learn: 0.9685663	total: 4m 5s	remaining: 5m 28s
171:	learn: 0.9685277	total: 4m 6s	remaining: 5m 26s
172:	learn: 0.9684820	total: 4m 8s	remaining: 5m 25s
173:	learn: 0.9684047	total: 4m 9s	remaining: 5m 24s
174:	learn: 0.9683062	total: 4m 11s

309:	learn: 0.9723870	total: 7m 27s	remaining: 2m 10s
310:	learn: 0.9723554	total: 7m 29s	remaining: 2m 8s
311:	learn: 0.9723202	total: 7m 31s	remaining: 2m 7s
312:	learn: 0.9722921	total: 7m 33s	remaining: 2m 5s
313:	learn: 0.9724081	total: 7m 34s	remaining: 2m 4s
314:	learn: 0.9724608	total: 7m 36s	remaining: 2m 3s
315:	learn: 0.9726506	total: 7m 37s	remaining: 2m 1s
316:	learn: 0.9726928	total: 7m 39s	remaining: 2m
317:	learn: 0.9727385	total: 7m 41s	remaining: 1m 58s
318:	learn: 0.9727631	total: 7m 42s	remaining: 1m 57s
319:	learn: 0.9727631	total: 7m 43s	remaining: 1m 55s
320:	learn: 0.9727702	total: 7m 45s	remaining: 1m 54s
321:	learn: 0.9727877	total: 7m 47s	remaining: 1m 53s
322:	learn: 0.9728334	total: 7m 48s	remaining: 1m 51s
323:	learn: 0.9727948	total: 7m 48s	remaining: 1m 49s
324:	learn: 0.9728686	total: 7m 50s	remaining: 1m 48s
325:	learn: 0.9728299	total: 7m 51s	remaining: 1m 47s
326:	learn: 0.9729283	total: 7m 53s	remaining: 1m 45s
327:	learn: 0.9729670	total: 7m 55s	re

In [15]:
#0.88997
!kaggle competitions submit -c skill-task -f submission_cat_modified_3.csv -m "Глубина 12, 400 итераций"


100%|██████████████████████████████████████| 5.89k/5.89k [00:00<00:00, 57.1kB/s]
Successfully submitted to Skill task